In [1]:
# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
import torch
print('hello')


c:\Users\Arjun\Desktop\AI_ENV\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


hello


In [2]:
torch.set_grad_enabled(False)
device = utils.get_device()
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

c:\Users\Arjun\Desktop\AI_ENV\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [3]:
model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h

In [29]:
dir(model)

['OV',
 'QK',
 'T_destination',
 'W_E',
 'W_E_pos',
 'W_K',
 'W_O',
 'W_Q',
 'W_U',
 'W_V',
 'W_gate',
 'W_in',
 'W_out',
 'W_pos',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_enable_hook',
 '_enable_hook_with_name',
 '_enable_hooks_for_points',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_init_weights_gpt2',
 '_in

In [4]:
loss = model('Java Joji rocks', return_type="loss")
loss.item()

11.13503646850586

In [5]:
loss = model('Java Joji rocks rocks rocks rocks rocks rocks ', return_type="loss")
loss.item()

7.309408664703369

In [6]:

loss = model('Java Joji rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rocks rock srocks rocks rocks rocks rocks ', return_type="loss")
loss.item()

2.715528726577759

In [7]:
text = 'Java joji rocks'
gpt2_tokens = model.to_tokens(text)

In [8]:
gpt2_logits, gpt2_cache = model.run_with_cache(gpt2_tokens, remove_batch_dim=True)

In [9]:
gpt2_logits

tensor([[[ 7.5261, 11.1214,  7.8919,  ..., -3.1299, -3.3873,  8.5934],
         [ 7.3828,  6.3855,  6.3817,  ..., -3.3074, -0.0224,  5.8700],
         [ 7.1220,  6.3180,  3.9884,  ..., -2.5630, -0.3930,  6.1892],
         [ 8.6471,  6.2821,  6.2502,  ..., -2.2384,  0.5549,  7.6160],
         [10.5680,  6.2048,  4.5058,  ..., -2.4789,  0.9009,  7.7411]]],
       device='cuda:0')

In [10]:
activations = {}

# Define a hook function that stores activations
def hook_fn(output):
    activations.append(output.detach().cpu())
    
# Add hooks to specific layers (for example, layers 0 and 1)
model.blocks[0].attn.hook_q.add_hook(lambda module, input, output: hook_fn(module, input, output, "layer_0_attn_q"))
model.blocks[0].mlp.hook_post.add_hook(lambda module, input, output: hook_fn(module, input, output, "layer_0_mlp_post"))
model.blocks[1].attn.hook_q.add_hook(lambda module, input, output: hook_fn(module, input, output, "layer_1_attn_q"))

# Input text
text = 'Java Joji rocks'

# Convert the input text into tokens
gpt2_tokens = model.to_tokens(text)

# Run the model and capture activations
output = model(gpt2_tokens)

# Now, the activations for the specified layers are stored in the dictionary
print("Activations from layer 0 attention q:", activations["layer_0_attn_q"].shape)
print("Activations from layer 0 MLP post:", activations["layer_0_mlp_post"].shape)
print("Activations from layer 1 attention q:", activations["layer_1_attn_q"].shape)

TypeError: <lambda>() got an unexpected keyword argument 'hook'